#### SAHIL BHATT  
#### Roll number : 2018111002  


**WORK DONE**:
1. Used Resnet-101 as pretrained model. Did not freeze any layers.  
2. Added a Dropout layer just before the final classification layer.  
3. Resized images to size 448x448.  
4. Did not normalize images. Trained on the entire training dataset.  
5. Augmented data by creating a dataset double the size of the original by horizontally flipping each image.  
6. Trained the model for 6 epochs at a learning rate of 1e-5, 2 epochs at a learning rate of 1e-6, and one epoch at a learning rate of 1e-7.  
7. Adam optimizer used, with cross-entropy loss.  

**RESULTS**:
Achieved the best F1-score (0.692) in the food recognition challenge, and significantly outperformed the previous best score of 0.676  

In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
import torchvision
import torch
import torch.nn as nn
import random
from PIL import Image, ImageEnhance
import torch.nn.functional as F
from torchvision import transforms
from torchvision.utils import make_grid
from torch.utils.data import random_split
from torchvision.transforms import ToTensor
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets.utils import download_url

df = pd.read_csv('train.csv')
le = LabelEncoder()
label_encoded = le.fit_transform(df['ClassName'])
label_names = le.classes_
df['label_encoded'] = label_encoded

flip_df = df.copy(deep=True)
df['fliplabel'] = [0]*len(df)
flip_df['fliplabel'] = [1]*len(flip_df)
merged_df = pd.concat([df,flip_df],ignore_index=True)


class FoodDataset(Dataset):
    def __init__(self, data_frame, root_dir,mode,transform=None):
        self.data_frame = data_frame
        self.root_dir = root_dir
        self.transform = transform
        self.mode = mode
    
    def __len__(self):
        return len(self.data_frame)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        img_name = os.path.join(self.root_dir, self.data_frame.iloc[idx, 0])
        image = Image.open(img_name)

        flip_value = self.data_frame.iloc[idx,-1]
        if flip_value == 1:
          image = image.transpose(Image.FLIP_LEFT_RIGHT)

        if self.mode=='test':
          label = None
        else:
          label = self.data_frame.iloc[idx, -2]
        
        if self.transform:
            image = self.transform(image)
    
        return (image, label)

food_train = FoodDataset(
    data_frame=merged_df,
    root_dir='image_data/train_images',
    mode = 'train',
    transform=transforms.Compose([
        transforms.Resize((448,448)), 
        transforms.ToTensor(),
        # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
)

def accuracy(outputs,labels):
  _, preds = torch.max(outputs,dim=1)
  return torch.tensor(torch.sum(preds==labels).item()/len(preds))



batch_size=16
train_dl=DataLoader(food_train, batch_size, shuffle=True, num_workers=4, pin_memory=True)


test_df = pd.read_csv('test.csv')

food_test = FoodDataset(
    data_frame=test_df,
    root_dir='image_data/test_images',
    mode = 'test',
    transform=transforms.Compose([
        transforms.Resize((448,448)), 
        transforms.ToTensor(),
        # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
)


test_dl = DataLoader(food_test, batch_size, num_workers=4, pin_memory=True)

from torchvision import models
modelresnet101 = models.resnet101(pretrained=True)
num_ftrs = modelresnet101.fc.in_features
modelresnet101.fc = nn.Sequential(
    nn.Dropout(0.4),
    nn.Linear(num_ftrs, 1000)
)

class ImageClassificationBase(nn.Module):
  def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  
        loss = F.cross_entropy(out, labels)
        return loss
    
  def epoch_end(self, epoch, result):
      print("Epoch [{}], train_loss: {:.4f}".format(epoch, result))

class ResnetModel(ImageClassificationBase):
  def __init__(self):
    super().__init__()
    self.network = modelresnet101
  def forward(self,x):
    return self.network(x)

resnet_model = ResnetModel()
resnet_model = nn.DataParallel(resnet_model)
# resnet_model.load_state_dict(torch.load('4482model_intel.pth'))
print("GPU numbers",torch.cuda.device_count())

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        print(device)
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)



if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

def to_device(data,device):
  if isinstance(data,(list,tuple)):
    return [to_device(x,device) for x in data]
  return data.to(device,non_blocking=True)

train_dl = DeviceDataLoader(train_dl, device)

def fit(epochs, lr, model, train_loader, opt_func=torch.optim.Adam):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.module.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        result = torch.stack(train_losses).mean().item()
        model.module.epoch_end(epoch, result)
        history.append(result)
    return history

resnet_model = to_device(resnet_model, device)
num_epochs = 6
opt_func = torch.optim.Adam
lr = 0.00001

history = fit(num_epochs, lr, resnet_model, train_dl, opt_func)
# torch.save(resnet_model.state_dict(), '4481model_intel.pth')
history = fit(2,0.000001,resnet_model,train_dl,opt_func)
# torch.save(resnet_model.state_dict(), '4482model_intel.pth')
history = fit(1,0.0000001,resnet_model,train_dl,opt_func)
# torch.save(resnet_model.state_dict(), '4483model_intel.pth')

def predict_single(input,model):
    input = to_device(input,device)
    inputs = input.unsqueeze(0)   
    model.eval()
    predictions = model(inputs)
    prediction = predictions[0].detach().cpu()
    return np.argmax(prediction)

pred_list = []
for img in food_test:
    pred = predict_single(img[0],resnet_model)
    predlabel = label_names[int(pred.item())]
    # print(predlabel)
    pred_list.append(predlabel)


# dictionary of lists  
dict = {'ClassName': pred_list}  
       
pred_df = pd.DataFrame(dict)
pred_df.to_csv('submission.csv',index=False)
